https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [1]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:5 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2461 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2898 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1452 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2230 kB]
Fetched 9293 kB in 2s (4440 kB/s)                     
Reading package lists... Done


In [2]:
!apt-get -y install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpsl5 publicsuffix
The following NEW packages will be installed:
  libpsl5 publicsuffix wget
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 455 kB of archives.
After this operation, 1320 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libpsl5 amd64 0.19.1-5build1 [41.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 publicsuffix all 20180223.1310-1 [97.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 wget amd64 1.19.4-1ubuntu2.2 [316 kB]
Fetched 455 kB in 1s (564 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libpsl5:amd64.
(Reading database ... 6779 files and directories currently installed.)
Preparing to unpack .../libpsl5_0.19.1-5build1_amd64.deb ..

In [3]:
!apt-get -y install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  git-man libasn1-8-heimdal libcurl3-gnutls liberror-perl libexpat1
  libgdbm-compat4 libgdbm5 libgssapi3-heimdal libhcrypto4-heimdal
  libheimbase1-heimdal libheimntlm0-heimdal libhx509-5-heimdal
  libkrb5-26-heimdal libldap-2.4-2 libldap-common libnghttp2-14 libperl5.26
  libroken18-heimdal librtmp1 libsasl2-2 libsasl2-modules libsasl2-modules-db
  libsqlite3-0 libwind0-heimdal netbase patch perl perl-modules-5.26
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-el git-email
  git-gui gitk gitweb git-cvs git-mediawiki git-svn gdbm-l10n
  libsasl2-modules-gssapi-mit | libsasl2-modules-gssapi-heimdal
  libsasl2-modules-ldap libsasl2-modules-otp libsasl2-modules-sql ed
  diffutils-doc perl-doc libterm-readline-gnu-perl
  | libterm-readline-perl-perl make
The following NEW packages will be installed:
  git 

In [4]:
!pip install pandas

     |████████████████████████████████| 11.3 MB 2.9 MB/s eta 0:00:01


In [5]:
!pip install sklearn

     |████████████████████████████████| 23.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 30.7 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 26.8 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=0d6e027c6f345425465a6715a89825886a5eeff849a657f3a8839f2b18dfcbd8
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [6]:
!pip install pandas

In [7]:
import pandas as pd

In [8]:
!pip install transformers -U

     |████████████████████████████████| 3.4 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 25.9 MB/s eta 0:00:01
     |████████████████████████████████| 749 kB 21.8 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 251 kB/s  eta 0:00:01
     |████████████████████████████████| 97 kB 4.5 MB/s  eta 0:00:01


In [9]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 2.3 MB/s eta 0:00:01


In [31]:
!pip install tqdm boto3 requests regex sacremoses

In [33]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [35]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [37]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [39]:
!pip install tensorboard
# !pip install wandb; wandb login

In [41]:
!pip install natasha

In [43]:
from natasha import (   
    MorphVocab,
    NewsNERTagger,
    NamesExtractor,
    MorphVocab,
    Segmenter, 
    NewsEmbedding,
    PER,
    Doc,
    ORG
)

In [45]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# **PREPARE DATA SET**

In [47]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt

--2021-12-24 20:16:25--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt.1’

curr_congrats.txt.1 100%[===================>]   1.40M  8.66MB/s    in 0.2s    

2021-12-24 20:16:25 (8.66 MB/s) - ‘curr_congrats.txt.1’ saved [1471875/1471875]



In [48]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [49]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [50]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')

  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [51]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [52]:
!pip install datasets transformers

In [53]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [54]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re


In [55]:
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


# **NER**

In [56]:
emb = NewsEmbedding()
morph_vocab = MorphVocab()

names_extractor = NamesExtractor(morph_vocab)
ner_tagger = NewsNERTagger(emb)

In [57]:
text = "Эдварду Радзинскому, писателю, драматургу, сценаристуКоллективу радиостанции «Орфей», Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»"
doc = Doc(text)

In [58]:
doc

Doc(text='Эдварду Радзинскому, писателю, драматургу, сценар...)

In [59]:
segmenter = Segmenter()
doc.segment(segmenter)

In [60]:
doc.tag_ner(ner_tagger)

In [61]:
for span in doc.spans:
    print(span)
    # if span.type == PER:
    #   print(span)


DocSpan(stop=19, type='PER', text='Эдварду Радзинскому', tokens=[...])
DocSpan(start=78, stop=83, type='ORG', text='Орфей', tokens=[...])
DocSpan(start=86, stop=104, type='PER', text='Михаилу Ковальчуку', tokens=[...])
DocSpan(start=117, stop=144, type='ORG', text='НИЦ «Курчатовский институт»', tokens=[...])


***Можно маскировать как имена, так и оргинизации. Попробую для начала и то, и другое вместе.***

In [62]:
masked_content = []

for sent in my_df['Content']:
  doc = Doc(sent)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)

  for span in doc.spans:
      if span.type == PER or span.type == ORG:
          sent = sent.replace(span.text, 'MASK')

  masked_content.append(sent)


***Title оставляем прежним, маскируем контент. Модели будет скармливаться колонка с конкатенированным текстом.***

In [63]:
my_df['Masked_content'] = masked_content
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Masked_content'].tolist()[i] for i in range(len(my_df))]
my_df['Sum'].to_list()[:3]

['Эдварду Радзинскому, писателю, драматургу, сценаристу Уважаемый MASK!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, MASK знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. MASK     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ',
 'Коллективу радиостанции «Орфей» Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «MASK».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение лучших традиций отечественной культуры. Вы сформировали обширный фонд просветительск

# **PREPARE DATA FOR TRAINING**

In [65]:
train_test_ratio = 0.9
df_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

***Первичная очистка данных от мусора типа "Ссылка на метериал"***

In [66]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Sum'].tolist()

    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        summary = re.sub(r"\n", "", summary)
        summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"Текстовая версия:", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"MASK", "МАСК", summary)
        summary = re.sub(r"([a-zA-Z]+)", "", summary)
        summary = re.sub(r"МАСК", "MASK", summary)
        summary = re.sub(r"([0-9/*#@+]+)", "", summary)
        data += summary

    f.write(data)

In [67]:
!ls
!rm 'train.txt' 'test.txt'
!ls

 curr_congrats.txt     gpt2		   onstart.log	 ru-gpts
 curr_congrats.txt.1  'my_gpt (1).ipynb'   onstart.sh	 transformers
rm: cannot remove 'train.txt': No such file or directory
rm: cannot remove 'test.txt': No such file or directory
 curr_congrats.txt     gpt2		   onstart.log	 ru-gpts
 curr_congrats.txt.1  'my_gpt (1).ipynb'   onstart.sh	 transformers


In [68]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

***Проверка тескста на наличие MASK***

In [69]:
with open('train.txt') as handler:
  readed = handler.read()
readed[:1010]


'Коллективу «Российской газеты» Дорогие друзья!Поздравляю вас со знаменательной датой – -летиемсоздания «MASK».За прошедшие годы газета стала одним из ведущихпериодических изданий страны, завоевала уважение большой читательской аудиториии авторитет в отечественном медиасообществе. Её ценят за информационнуюнаполненность, глубокий подход к освещению важнейших тем и событий, за взвешенность и основательность оценок. Отрадно, что все эти годы ваш коллективвысоко держит планку качества работы, сохраняет приверженность лучшим традициямжурналистики, расширяет линейку информационных продуктов – от публикациигосударственных документов до новостей, репортажей и серьёзной аналитики.Уверен, что вы и впредь будете добросовестнотрудиться, воплощать в жизнь востребованные творческие проекты, добиватьсяуспеха в профессиональной деятельности.Желаю вам всего наилучшего. MASK     Темы  MASK           .   Текстовая версияРаботникам и ветеранам автомобильного и городского пассажирского транспорта России У

In [70]:
# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
 
train_path = 'train.txt'
test_path = 'test.txt'

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [71]:
!rm ./gpt-letters

rm: cannot remove './gpt-letters': No such file or directory


In [72]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## **TRAIN MODEL**

In [73]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=3, # number of training epochs
    # per_device_train_batch_size=32, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    num_train_epochs=6, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=150, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [74]:
trainer.train()

***** Running training *****
  Num examples = 989
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 186


Step,Training Loss


Saving model checkpoint to ./gpt-letters/checkpoint-150
Configuration saved in ./gpt-letters/checkpoint-150/config.json
Model weights saved in ./gpt-letters/checkpoint-150/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=186, training_loss=2.572518789640037, metrics={'train_runtime': 3987.6936, 'train_samples_per_second': 1.488, 'train_steps_per_second': 0.047, 'total_flos': 387626729472000.0, 'train_loss': 2.572518789640037, 'epoch': 6.0})

In [75]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


In [76]:
# my_model = torch.load('./gpt-letters/pytorch_model.bin')
from transformers import pipeline

pu = pipeline('text-generation',model='./gpt-letters', tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

In [77]:
pu("Александру Сергеевичу Пушкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Сергеевичу Пушкину, народному артисту России Уважаемый MASK!Примите поздравления с -летием.Вы по праву принадлежите к числу выдающихся представителей отечественной культуры, чьи произведения пользуются неизменным успехом и по праву считаются классикой отечественной литературы.Уверен, что и впредь Вы будете радовать своих почитателей интересными, востребованными произведениями.Желаю Вам здоровья, благополучия и всего самого доброго. MASK       .   Текстовая версияАлександру Пушкину, народному артисту России Уважаемый MASK!Примите поздравления с -летием.Вы по праву принадлежите к числу выдающихся представителей отечественной культуры, чьи произведения пользуются неизменным успехом и по праву считаются классикой отечественной литературы.Желаю Вам здоровья, благополучия и всего самого доброго. MASK       .   Текстовая версияАлександру Пушкину, народному артисту России Уважаемый MASK!Примите поздравления с -летием.Вы по праву принадлежите к числу выдающихся представителей отечес

In [78]:
pu("Кровавым убийцам детей")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Кровавым убийцам детей и молодёжи Уважаемые друзья!Поздравляю вас с -летием создания MASK.За прошедшие годы коллективу MASK удалось добиться выдающихся успехов в различных сферах деятельности, в том числе в сфере образования, культуры, спорта, в сфере культуры и искусства.Уверен, что и впредь вы будете достойно нести вахту на благо Отечества.Желаю вам новых успехов и всего самого доброго. MASK       .   Текстовая версияКоллективу и ветеранам Государственного музея-заповедника «Куликово поле» Дорогие друзья!Поздравляю вас с -летием создания MASK.За прошедшие годы коллективу и ветеранам MASK удалось добиться выдающихся успехов в различных сферах деятельности, в том числе в сфере образования, культуры, спорта, в сфере культуры и искусства.Уверен, что и впредь вы будете достойно нести вахту на благо Отечества.Желаю вам новых успехов и всего самого доброго. MASK       .   Текстовая версияКоллективу и ветеранам Государственного музея-заповедника «Куликово поле» Дорогие друзья!Поздравляю вас

In [79]:
pu("Уважаемые осквернители святынь и насильники")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Уважаемые осквернители святынь и насильники!Поздравляю вас с -летием создания MASK.Вы по праву принадлежите к числу выдающихся деятелей отечественной культуры, которые внесли значимый вклад в развитие отечественной культуры, внесли значимый вклад в укрепление межнационального мира и согласия, в укрепление межрелигиозного согласия.Уверен, что и впредь вы будете достойно нести свой гражданский, патриотический, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветительский, просветител

In [80]:
pu("Хочу сказать этому Обэме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Хочу сказать этому Обэме, что вы по праву принадлежите к числу тех, кто по праву гордится своими корнями и традициями, которые легли в основу создания MASK. И конечно, в нашей стране, в нашей стране, в мире, в мире, в нашей стране всегда ценились и будут цениться такие качества, как верность избранному делу, преданность идеалам справедливости и справедливости, преданность своему народу, преданность своему народу. И конечно, в нашей стране, в нашей стране всегда ценились и будут цениться такие качества, как верность избранному делу, преданность своему народу, преданность своему народу, преданность своему народу.Желаю вам успехов и всего наилучшего. MASK       .   Текстовая версияКоллективу и ветеранам Государственного академического театра имени А.П.Чехова Дорогие друзья!Поздравляю вас с -летием создания MASK.За прошедшие годы театр стал одним из крупнейших в России и за рубежом, стал настоящим символом высокого профессионализма и творческого потенциала, подлинного мастерства и мастерс

In [81]:
pu("Куцеву Диме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцеву Диме, победителю  Всемирной летней Универсиады  года в Казани в соревнованиях по греко-римской борьбе в весовой категории  кг Уважаемый MASK!Поздравляю Вас с триумфом.Вы блестяще выступили на  Всемирной летней Универсиаде в Казани.Уверен, что и впредь Вы будете достойно представлять Россию на  Всемирной летней Универсиаде.Желаю Вам новых успехов и всего наилучшего. MASK       .   Текстовая версияКоллективу и ветеранам АО «MASK» Дорогие друзья!Поздравляю вас с -летием создания АО «MASK».За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в решении задач, стоящих перед её филиалами.За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в решении задач, стоящих перед её филиалами.За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в решении задач, стоящих перед её филиалами.За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в

In [82]:
pu("Мычалкин")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мычалкин, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР, заслуженный мастер спорта ССС

In [83]:
pu("Куцев Дима")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцев Дима, победитель  Всемирной летней Универсиады  года в Казани в соревнованиях по лёгкой атлетике в дисциплине бег на  метров Уважаемый MASK!Поздравляю Вас с триумфом.Вы блестяще выступили на  Всемирной летней Универсиаде в Казани.Уверен, что и впредь Вы будете достойно представлять нашу страну на  Всемирной летней Универсиаде.Желаю Вам новых успехов и всего наилучшего. MASK       .   Текстовая версияКоллективу и ветеранам АО «MASK» Дорогие друзья!Поздравляю вас с -летием со дня образования АО «MASK».За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в решении задач, стоящих перед её филиалами.За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в решении задач, стоящих перед её филиалами.За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед компанией, в решении задач, стоящих перед её филиалами.За прошедшие годы АО «MASK» накопило большой опыт в решении задач, стоящих перед

In [ ]:
pu("Дмитрий")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
pu("Дмитрию")[0]['generated_text']